In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
with open("/content/India, officially the Republic of I.txt",'r',encoding='utf-8') as f:
    raw_text=f.read()
print(len(raw_text))
print(raw_text[:100])

55955
India, officially the Republic of India,[j][21] is a country in South Asia. It is the seventh-larges


In [ ]:
print("Characters:",len(raw_text))
print("Total_tokens:",len(tokenizer.encode(raw_text)))

Characters: 55955
Total_tokens: 12860


In [ ]:
train_ratio = 0.9
train_size = int(train_ratio * len(raw_text))
train_text = raw_text[:train_size]
val_text = raw_text[train_size:]

In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.7 MB/s eta 0:00:00


In [ ]:
from torch.utils.data import Dataset, DataLoader
import tiktoken

class GPTTokenizerDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
        token_ids = self.tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    dataset = GPTTokenizerDataset(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last
    )
    return dataloader

In [ ]:
def generate_text(model,idx,max_new_tokens,context_size):
    for _ in range(max_new_tokens):
        idx_cond=idx[:,-context_size:]
        with torch.no_grad():
            logits=model(idx_cond)
        logits=logits[:,-1,:]
        probas=torch.softmax(logits,dim=-1)
        idx_next=torch.argmax(probas,dim=-1,keepdim=True)
        idx=torch.cat((idx,idx_next),dim=1)
    return idx

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")
def text_to_token_ids(text,tokenizer):
    encoded=tokenizer.encode(text,allowed_special={'<|endoftext|>'})
    encoded_tensor=torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor
def token_ids_to_text(token_ids,tokenizer):
    flat=token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())

Coding up the Attention model:- Here we would be creating a class of the causal attention and instantiating multiple times for the multihead attention model.

Now for example if we set the number of heads we want is 10, then what exactly happens:-
--> we obtain a tensor with ten sets of context vector matrices.
--> In each context vector matrix the rows represent the context vectors corresponding to the tokens, and the columns corresponding to the embedding dimension specified via d_out.
--> Final embedding dimension is 10 x 10.

IMPLEMENTING THE PARALLEL METHOD OF IMPLEMENTATION.

In [ ]:
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec

In [ ]:
#Defining the parameters
GPT_CONFIG={
    'vocab_size':50257,
    'context_length':256, # Change it to 1024 or greater if we have gpu
    'embedding_dim':512,
    'num_heads':16,
    'n_layers':12,
    'dropout':0.1,
    'qkv_bias':False #Whether to include a bias layer in the linear layers of the multi head attention for query,key and value computations.
}

Coding up the placeholder architecture, it is like the mothership from where all the robots will branch out

In [ ]:
class GPT_Model(nn.Module):
    def __init__(self, cfg):
        #The __init__ constructor of this GPTModel class initializes the token and positional embedding layers using the configurations passed in via a Python dictionary, cfg.
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["embedding_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["embedding_dim"])
        self.drop_emb = nn.Dropout(cfg["dropout"])
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.final_norm = LayerNormalization(cfg["embedding_dim"])
        self.out_head = nn.Linear(cfg["embedding_dim"], cfg["vocab_size"], bias=False)
    def forward(self,in_idx):
        batch_size,seq_len=in_idx.shape
        #in_idx = torch.clamp(in_idx, 0, self.tok_emb.num_embeddings - 1)
        token_embeddings=self.tok_emb(in_idx)
        positions = torch.arange(seq_len, device=in_idx.device).unsqueeze(0) #this is the extra added line
        positional_embeddings=self.pos_emb(positions)
        x=token_embeddings+positional_embeddings
        x=self.drop_emb(x)
        x=self.trf_blocks(x)
        x=self.final_norm(x)
        logits=self.out_head(x)
        return logits

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.att=MultiHeadAttention(
            d_in=config["embedding_dim"],
            d_out=config["embedding_dim"],
            context_length=config['context_length'],
            dropout=config['dropout'],
            num_heads=config['num_heads'],
            qkv_bias=config['qkv_bias']
        )
        self.ff=FeedForward(config)
        self.norm1=LayerNormalization(config["embedding_dim"])
        self.norm2=LayerNormalization(config["embedding_dim"])
        self.drop_resid=nn.Dropout(config['dropout'])
    def forward(self,x):
        shortcut=x
        x=self.norm1(x)
        x=self.att(x)
        x=self.drop_resid(x)
        x=x+shortcut
        shortcut=x
        x=self.norm2(x)
        x=self.ff(x)
        x=self.drop_resid(x)
        x=x+shortcut
        return x

In [ ]:
class LayerNormalization(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    def forward(self,x):
        mean= x.mean(-1, keepdim=True)
        variance = x.var(-1, keepdim=True)
        norm_x=(x-mean)/(torch.sqrt(variance+self.eps))
        return self.scale*norm_x + self.shift

We will use swish activation function.

In [ ]:
class Swish(nn.Module):
    def __init__(self):
        super(Swish, self).__init__()
    def forward(self, x):
        return x * torch.sigmoid(x)

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layers=nn.Sequential(
            nn.Linear(config["embedding_dim"], 4*config["embedding_dim"]),
            Swish(),
            nn.Linear(4*config["embedding_dim"], config["embedding_dim"]),
        )
    def forward(self, x):
        return self.layers(x)

In [ ]:
class DeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes,use_shortcut):
        super().__init__()
        self.layers=nn.ModuleList([
            #We would be implementing 10 layers
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1])),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2])),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3])),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4])),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5])),
            nn.Sequential(nn.Linear(layer_sizes[5], layer_sizes[6])),
            nn.Sequential(nn.Linear(layer_sizes[6], layer_sizes[7])),
            nn.Sequential(nn.Linear(layer_sizes[7], layer_sizes[8])),
            nn.Sequential(nn.Linear(layer_sizes[8], layer_sizes[9])),
            nn.Sequential(nn.Linear(layer_sizes[9], layer_sizes[10])),
        ])
    def forward(self,x):
        for layer in self.layers:
            #Computing the output of the current layer
            layer_output=layer(x)
            #Check if shortcut can be applied
            if self.use_shortcut and x.shape==layer_output.shape:
                x=x+layer_output
            else:
                x=layer_output
            return x
def print_gradients(model,x):
    #First would be the forward pass
    output = model(x)
    target=torch.tensor([0,])
    #Loss calculation
    loss=nn.MSELoss()
    loss=loss(output,target)
    loss.backward()
    for name, param in model.named_parameters():
        if 'weight' in name:
            print(f"{name} grad: {param.grad}")

Now let us initialise

In [ ]:
batch_size = 2  # Number of samples in the batch
sequence_length = 10  # Length of each sequence
vocab_size = 100  # Size of the vocabulary
batch = torch.randint(0, vocab_size, (batch_size, sequence_length))
print(batch)

tensor([[44, 53, 61, 15,  9, 70, 67, 46, 79,  5],
        [76, 60, 56, 92, 14, 14, 53, 15, 44, 27]])


In [ ]:
torch.manual_seed(123)
model=GPT_Model(GPT_CONFIG)
out=model(batch)
print("Input batch:\n",batch)
print("Output batch:\n",out.shape)
print(out)

Input batch:
 tensor([[44, 53, 61, 15,  9, 70, 67, 46, 79,  5],
        [76, 60, 56, 92, 14, 14, 53, 15, 44, 27]])
Output batch:
 torch.Size([2, 10, 50000])
tensor([[[ 0.4678, -0.5511, -0.5683,  ...,  1.4707,  1.3414,  0.3417],
         [ 0.7936,  0.7604,  0.0037,  ..., -0.1304,  0.0257,  0.4613],
         [ 0.4322,  0.3863, -0.3351,  ..., -0.5346,  1.2010,  0.4920],
         ...,
         [ 0.8807, -0.1948, -0.3523,  ..., -0.4180,  0.6709, -0.4552],
         [ 0.2633,  0.5850,  0.1626,  ..., -1.2413,  0.3800,  0.1911],
         [ 0.9222,  0.3679, -0.4887,  ...,  0.3828,  0.5291,  1.5582]],

        [[ 0.6132, -0.1044,  0.3924,  ...,  0.6008,  0.3865,  0.8070],
         [ 0.5212,  0.3173,  0.0860,  ...,  0.1296,  0.2347,  0.3906],
         [ 0.0695, -0.3108, -0.2167,  ...,  0.3888,  0.4980, -0.1511],
         ...,
         [-0.3946, -0.0402,  0.5788,  ..., -0.0052,  0.2615,  0.1750],
         [-0.1209,  0.1735,  0.3235,  ..., -0.9687,  0.3305, -0.2390],
         [ 0.6061,  0.4015,  0.3

Displaying the number of parameters for the GPT model

In [ ]:
total_parameters=sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_parameters}")
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

Total number of parameters: 89142272
Token embedding layer shape: torch.Size([50000, 512])
Output layer shape: torch.Size([50000, 512])


Number of trainable parameters in the model

In [ ]:
total_params_gpt2 = total_parameters - sum(p.numel() for p in model.out_head.parameters())
print(f"Number of trainable parameters considering weight tying: {total_params_gpt2}")

Number of trainable parameters considering weight tying: 63542272


In [ ]:
total_size_in_bytes=total_parameters*4

total_size_of_the_model_in_MB=total_size_in_bytes/(1024*1024)
print(f"Total size of the model : {total_size_of_the_model_in_MB:.2f} MB")

Total size of the model : 340.05 MB


Total size of the model : 341.55 MB
Number of trainable parameters considering weight tying: 63935488


The next step is to now decode these tensors to proper text. Which would be coding up in the subsequent steps

In [ ]:
#Let us try out the decoding procedure
start_context="Hello, I am Aditya."
tokenizer = tiktoken.get_encoding("cl100k_base")
encoded=tokenizer.encode(start_context)
print(encoded)

[9906, 11, 358, 1097, 2467, 488, 64, 13]


In [ ]:
model.eval()

GPT_Model(
  (tok_emb): Embedding(50000, 512)
  (pos_emb): Embedding(256, 512)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=512, out_features=512, bias=False)
        (W_key): Linear(in_features=512, out_features=512, bias=False)
        (W_value): Linear(in_features=512, out_features=512, bias=False)
        (out_proj): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): Swish()
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
      )
      (norm1): LayerNormalization()
      (norm2): LayerNormalization()
      (drop_resid): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): L

In [ ]:
model.eval()
out=generate_text(model=model,idx=torch.tensor(encoded).unsqueeze(0),max_new_tokens=6,context_size=GPT_CONFIG["context_length"])
print("Output:\n",out)

Output:
 tensor([[ 9906,    11,   358,  1097,  2467,   488,    64,    13,  2163,  8019,
          7580, 20329, 32731,    25]])


In [ ]:
start_context="Hello, I am Aditya I want to become a CEO one day of my own company"
token_ids=generate_text(model=model,idx=text_to_token_ids(start_context,tokenizer),max_new_tokens=10,context_size=GPT_CONFIG["context_length"])
print("Output text:\n",token_ids_to_text(token_ids,tokenizer))

Output text:
 Hello, I am Aditya I want to become a CEO one day of my own companyPub.bukkitregistry(fields [[ ErrorMessageoptional Send thereleg


In [ ]:
inputs=torch.tensor([[ 9906,    11,   358,  1097,  2467,   488,    64,    13, 41867, 40540,
         15145, 30876, 46468, 30001]])  # Remove extra comma and parenthesis to make it a tensor
with torch.no_grad():
  logits=model(inputs)
probas=torch.softmax(logits,dim=-1)
print(probas.shape)

torch.Size([1, 14, 50000])


In [ ]:
torch.manual_seed(123)
train_loader=create_dataloader_v1(train_text,batch_size=2,max_length=GPT_CONFIG["context_length"],
                                  stride=GPT_CONFIG['context_length'],
                                  drop_last=True,
                                  shuffle=True
                                  )
val_loader=create_dataloader_v1(val_text,batch_size=2,max_length=GPT_CONFIG["context_length"],
                                  stride=GPT_CONFIG['context_length'],
                                  drop_last=True,
                                  shuffle=True
                                  )

In [ ]:
def calculation_of_loss(input_batch,target_batch,model,device):
  input_batch,target_batch=input_batch.to(device),target_batch.to(device)
  logits=model(input_batch)
  loss=torch.nn.functional.cross_entropy(logits.flatten(0,1),target_batch.flatten())
  return loss

In [ ]:
def loss_loader(data_loader,model,device,num_batches=10):
  total_loss=0
  num_batches=min(num_batches,len(data_loader))
  for i,(input_batch,target_batch) in enumerate(data_loader):
    if i<num_batches:
      loss=calculation_of_loss(input_batch,target_batch,model,device)
      total_loss+=loss.item()
    else:
      break
  return total_loss/num_batches

In [ ]:
device='cpu'
model.to(device)
train_loss=loss_loader(train_loader,model,device)
val_loss=loss_loader(val_loader,model,device)
print(f"Train loss: {train_loss:.4f}")
print(f"Validation loss: {val_loss:.4f}")

IndexError: index out of range in self